# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 19 2022 2:11PM,246739,10,0085993271,ISDIN Corporation,Released
1,Sep 19 2022 1:53PM,246738,10,Enova-10659,Emerginnova,Released
2,Sep 19 2022 1:53PM,246738,10,Enova-10660,Emerginnova,Released
3,Sep 19 2022 1:53PM,246738,10,Enova-10661,Emerginnova,Released
4,Sep 19 2022 1:53PM,246738,10,Enova-10662,Emerginnova,Released
5,Sep 19 2022 1:53PM,246738,10,Enova-10663,Emerginnova,Released
6,Sep 19 2022 1:53PM,246738,10,Enova-10664,Emerginnova,Released
7,Sep 19 2022 1:51PM,246737,12,HH6696.2,Hush Hush,Executing
8,Sep 19 2022 1:49PM,246736,10,0085993188,ISDIN Corporation,Released
9,Sep 19 2022 1:49PM,246736,10,0085993187,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
21,246735,Released,1
22,246736,Released,22
23,246737,Executing,1
24,246738,Released,6
25,246739,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246735,NaN,NaN,1.0
246736,NaN,NaN,22.0
246737,NaN,1.0,NaN
246738,NaN,NaN,6.0
246739,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246677,0.0,1.0,0.0
246693,0.0,1.0,0.0
246700,2.0,11.0,5.0
246701,0.0,0.0,2.0
246708,0.0,8.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246677,0,1,0
246693,0,1,0
246700,2,11,5
246701,0,0,2
246708,0,8,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246677,0,1,0
1,246693,0,1,0
2,246700,2,11,5
3,246701,0,0,2
4,246708,0,8,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246677,,1,
1,246693,,1,
2,246700,2,11,5
3,246701,,,2
4,246708,,8,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 19 2022 2:11PM,246739,10,ISDIN Corporation
1,Sep 19 2022 1:53PM,246738,10,Emerginnova
7,Sep 19 2022 1:51PM,246737,12,Hush Hush
8,Sep 19 2022 1:49PM,246736,10,ISDIN Corporation
30,Sep 19 2022 1:44PM,246734,10,ISDIN Corporation
49,Sep 19 2022 1:44PM,246735,19,"GUSA Granules USA, Inc."
50,Sep 19 2022 1:39PM,246733,10,Yusen – 3D Matrix
51,Sep 19 2022 1:31PM,246732,19,"GUSA Granules USA, Inc."
52,Sep 19 2022 1:31PM,246731,10,Yusen – 3D Matrix
53,Sep 19 2022 1:30PM,246730,10,"Senseonics, Incorporated"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 19 2022 2:11PM,246739,10,ISDIN Corporation,,,1
1,Sep 19 2022 1:53PM,246738,10,Emerginnova,,,6
2,Sep 19 2022 1:51PM,246737,12,Hush Hush,,1,
3,Sep 19 2022 1:49PM,246736,10,ISDIN Corporation,,,22
4,Sep 19 2022 1:44PM,246734,10,ISDIN Corporation,,,19
5,Sep 19 2022 1:44PM,246735,19,"GUSA Granules USA, Inc.",,,1
6,Sep 19 2022 1:39PM,246733,10,Yusen – 3D Matrix,,,1
7,Sep 19 2022 1:31PM,246732,19,"GUSA Granules USA, Inc.",,,1
8,Sep 19 2022 1:31PM,246731,10,Yusen – 3D Matrix,,,1
9,Sep 19 2022 1:30PM,246730,10,"Senseonics, Incorporated",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 19 2022 2:11PM,246739,10,ISDIN Corporation,1,,
1,Sep 19 2022 1:53PM,246738,10,Emerginnova,6,,
2,Sep 19 2022 1:51PM,246737,12,Hush Hush,,1,
3,Sep 19 2022 1:49PM,246736,10,ISDIN Corporation,22,,
4,Sep 19 2022 1:44PM,246734,10,ISDIN Corporation,19,,
5,Sep 19 2022 1:44PM,246735,19,"GUSA Granules USA, Inc.",1,,
6,Sep 19 2022 1:39PM,246733,10,Yusen – 3D Matrix,1,,
7,Sep 19 2022 1:31PM,246732,19,"GUSA Granules USA, Inc.",1,,
8,Sep 19 2022 1:31PM,246731,10,Yusen – 3D Matrix,1,,
9,Sep 19 2022 1:30PM,246730,10,"Senseonics, Incorporated",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 19 2022 2:11PM,246739,10,ISDIN Corporation,1,,
1,Sep 19 2022 1:53PM,246738,10,Emerginnova,6,,
2,Sep 19 2022 1:51PM,246737,12,Hush Hush,,1,
3,Sep 19 2022 1:49PM,246736,10,ISDIN Corporation,22,,
4,Sep 19 2022 1:44PM,246734,10,ISDIN Corporation,19,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 19 2022 2:11PM,246739,10,ISDIN Corporation,1.0,NaN,NaN
1,Sep 19 2022 1:53PM,246738,10,Emerginnova,6.0,NaN,NaN
2,Sep 19 2022 1:51PM,246737,12,Hush Hush,NaN,1.0,NaN
3,Sep 19 2022 1:49PM,246736,10,ISDIN Corporation,22.0,NaN,NaN
4,Sep 19 2022 1:44PM,246734,10,ISDIN Corporation,19.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 19 2022 2:11PM,246739,10,ISDIN Corporation,1.0,0.0,0.0
1,Sep 19 2022 1:53PM,246738,10,Emerginnova,6.0,0.0,0.0
2,Sep 19 2022 1:51PM,246737,12,Hush Hush,0.0,1.0,0.0
3,Sep 19 2022 1:49PM,246736,10,ISDIN Corporation,22.0,0.0,0.0
4,Sep 19 2022 1:44PM,246734,10,ISDIN Corporation,19.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2713968,66.0,19.0,2.0
12,246737,0.0,1.0,0.0
15,246712,64.0,0.0,0.0
16,986817,2.0,2.0,0.0
19,1233636,6.0,23.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2713968,66.0,19.0,2.0
1,12,246737,0.0,1.0,0.0
2,15,246712,64.0,0.0,0.0
3,16,986817,2.0,2.0,0.0
4,19,1233636,6.0,23.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,66.0,19.0,2.0
1,12,0.0,1.0,0.0
2,15,64.0,0.0,0.0
3,16,2.0,2.0,0.0
4,19,6.0,23.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,66.0
1,12,Released,0.0
2,15,Released,64.0
3,16,Released,2.0
4,19,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19
Status,,,,,
Completed,2.0,0.0,0.0,0.0,0.0
Executing,19.0,1.0,0.0,2.0,23.0
Released,66.0,0.0,64.0,2.0,6.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19
0,Completed,2.0,0.0,0.0,0.0,0.0
1,Executing,19.0,1.0,0.0,2.0,23.0
2,Released,66.0,0.0,64.0,2.0,6.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19
0,Completed,2.0,0.0,0.0,0.0,0.0
1,Executing,19.0,1.0,0.0,2.0,23.0
2,Released,66.0,0.0,64.0,2.0,6.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()